# 🔍 TRM POC - Notebook 2: RAG Embeddings Generation

**Objectif:** Générer les embeddings sémantiques des corpus (Spinoza/Bergson/Kant) avec sentence-transformers

**Runtime:** GPU Colab gratuit (T4 - 12h max)

**Durée estimée:** 1-2h (génération + indexation)

---

## Phase 0 - POC TRM (0€)

Ce notebook implémente:
1. Upload corpus philosophiques (Spinoza/Bergson/Kant)
2. Découpage en passages (sections Markdown)
3. Génération embeddings (sentence-transformers)
4. Indexation FAISS pour retrieval rapide
5. Export index pour réutilisation (Notebook 3 + Vast.ai)

**Note:** GPU T4 gratuit suffisant pour embeddings (modèle léger)

## 1. Installation Dépendances

In [ ]:
# Installation des librairies nécessaires
!pip install -q sentence-transformers faiss-gpu

print("✅ Dépendances installées")

## 2. Upload Corpus Fichiers

**Action manuelle:** Uploader les fichiers suivants depuis `bergsonAndFriends/data/RAG/`:
- `Corpus Spinoza Dialogique 18k - Éthique II-IV.md`
- `Glossaire Conversationnel Spinoza - 12 Concepts.md`
- `corpus_bergson_27k_dialogique.md`
- `glossaire_bergson_conversationnel.md`
- `corpus_kant_20k.txt.md`
- `glossaire_kant_conversationnel.md`

Ou cloner le repo directement:

In [ ]:
# Option 1: Upload manuel via interface Colab
from google.colab import files
print("📤 Upload fichiers corpus (6 fichiers .md attendus)")
uploaded = files.upload()

# Option 2: Clone repo (si public)
# !git clone https://github.com/YOUR_USERNAME/bergsonAndFriends.git
# CORPUS_DIR = "bergsonAndFriends/data/RAG/"

## 3. Configuration & Imports

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
import pickle
from pathlib import Path
from typing import List, Dict
import re

# Configuration
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # Léger, rapide, multilingue
# Alternative: "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

CORPUS_FILES = {
    "spinoza": {
        "corpus": "Corpus Spinoza Dialogique 18k - Éthique II-IV.md",
        "glossaire": "Glossaire Conversationnel Spinoza - 12 Concepts.md"
    },
    "bergson": {
        "corpus": "corpus_bergson_27k_dialogique.md",
        "glossaire": "glossaire_bergson_conversationnel.md"
    },
    "kant": {
        "corpus": "corpus_kant_20k.txt.md",
        "glossaire": "glossaire_kant_conversationnel.md"
    }
}

print("✅ Configuration OK")

## 4. Chargement Modèle Embeddings

In [ ]:
print(f"⏳ Chargement {EMBEDDING_MODEL}...")
embedder = SentenceTransformer(EMBEDDING_MODEL)
print(f"✅ Modèle chargé - Dimension: {embedder.get_sentence_embedding_dimension()}")

## 5. Parsing Corpus en Passages

In [ ]:
def parse_corpus_to_passages(text: str, philosopher: str) -> List[Dict]:
    """
    Découpe corpus markdown en passages (basé sur headers ##).
    
    Returns:
        Liste de passages avec metadata
    """
    passages = []
    lines = text.split('\n')
    current_section = {"title": "", "content": ""}
    
    for line in lines:
        if line.startswith('##'):
            # Sauvegarder section précédente
            if current_section["content"].strip():
                passages.append({
                    "text": f"{current_section['title']}\n{current_section['content']}",
                    "title": current_section["title"],
                    "philosopher": philosopher,
                    "type": "corpus"
                })
            # Nouvelle section
            current_section = {
                "title": re.sub(r'^#+\s*', '', line),
                "content": ""
            }
        else:
            current_section["content"] += line + '\n'
    
    # Dernière section
    if current_section["content"].strip():
        passages.append({
            "text": f"{current_section['title']}\n{current_section['content']}",
            "title": current_section["title"],
            "philosopher": philosopher,
            "type": "corpus"
        })
    
    return passages

print("✅ Fonction parsing définie")

## 6. Génération Embeddings Par Philosophe

In [ ]:
# Stockage passages + embeddings par philosophe
all_data = {}

for philosopher, files in CORPUS_FILES.items():
    print(f"\n{'='*60}")
    print(f"📚 Traitement: {philosopher.upper()}")
    print(f"{'='*60}")
    
    passages = []
    
    # Charger corpus
    corpus_file = files["corpus"]
    if Path(corpus_file).exists():
        with open(corpus_file, 'r', encoding='utf-8') as f:
            corpus_text = f.read()
        corpus_passages = parse_corpus_to_passages(corpus_text, philosopher)
        passages.extend(corpus_passages)
        print(f"  ✅ Corpus: {len(corpus_passages)} passages")
    
    # Charger glossaire
    glossaire_file = files["glossaire"]
    if Path(glossaire_file).exists():
        with open(glossaire_file, 'r', encoding='utf-8') as f:
            glossaire_text = f.read()
        glossaire_passages = parse_corpus_to_passages(glossaire_text, philosopher)
        for p in glossaire_passages:
            p["type"] = "glossaire"  # Marquer glossaire
        passages.extend(glossaire_passages)
        print(f"  ✅ Glossaire: {len(glossaire_passages)} passages")
    
    print(f"  📊 Total passages: {len(passages)}")
    
    # Générer embeddings
    print(f"  ⏳ Génération embeddings...")
    texts = [p["text"] for p in passages]
    embeddings = embedder.encode(texts, show_progress_bar=True, convert_to_numpy=True)
    print(f"  ✅ Embeddings générés: {embeddings.shape}")
    
    # Stocker
    all_data[philosopher] = {
        "passages": passages,
        "embeddings": embeddings
    }

print(f"\n✅ Tous les embeddings générés !")
for phil, data in all_data.items():
    print(f"  {phil}: {len(data['passages'])} passages")

## 7. Indexation FAISS

In [ ]:
# Créer index FAISS par philosophe
faiss_indexes = {}

for philosopher, data in all_data.items():
    print(f"\n🔍 Indexation FAISS: {philosopher}")
    
    embeddings = data["embeddings"]
    dimension = embeddings.shape[1]
    
    # Créer index FAISS (IndexFlatIP = Inner Product, bon pour cosine similarity)
    index = faiss.IndexFlatIP(dimension)
    
    # Normaliser embeddings (pour cosine similarity)
    faiss.normalize_L2(embeddings)
    
    # Ajouter à l'index
    index.add(embeddings)
    
    faiss_indexes[philosopher] = index
    print(f"  ✅ Index créé: {index.ntotal} vecteurs")

print("\n✅ Tous les index FAISS créés !")

## 8. Test Retrieval

In [ ]:
def retrieve_passages(query: str, philosopher: str, top_k: int = 3):
    """
    Retrieve passages pertinents via FAISS.
    """
    # Encoder query
    query_emb = embedder.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_emb)
    
    # Recherche
    index = faiss_indexes[philosopher]
    scores, indices = index.search(query_emb, top_k)
    
    # Récupérer passages
    results = []
    for score, idx in zip(scores[0], indices[0]):
        passage = all_data[philosopher]["passages"][idx]
        passage["similarity_score"] = float(score)
        results.append(passage)
    
    return results

# Test
print("\n🧪 TEST RETRIEVAL")
print("="*60)
test_query = "Qu'est-ce que le conatus ?"
results = retrieve_passages(test_query, "spinoza", top_k=3)

print(f"Query: '{test_query}'")
print(f"\nTop 3 passages (Spinoza):\n")
for i, r in enumerate(results, 1):
    print(f"{i}. [{r['type']}] {r['title']} (score: {r['similarity_score']:.3f})")
    print(f"   {r['text'][:150]}...\n")

print("✅ Retrieval fonctionnel !")

## 9. Export pour Réutilisation

In [ ]:
# Créer dossier export
!mkdir -p /content/rag_exports

# Sauvegarder index FAISS + passages par philosophe
for philosopher in all_data.keys():
    print(f"\n💾 Export: {philosopher}")
    
    # Index FAISS
    faiss.write_index(faiss_indexes[philosopher], f"/content/rag_exports/{philosopher}_faiss.index")
    print(f"  ✅ FAISS index saved")
    
    # Passages (pickle)
    with open(f"/content/rag_exports/{philosopher}_passages.pkl", 'wb') as f:
        pickle.dump(all_data[philosopher]["passages"], f)
    print(f"  ✅ Passages saved")

# Sauvegarder config
config = {
    "embedding_model": EMBEDDING_MODEL,
    "dimension": embedder.get_sentence_embedding_dimension(),
    "philosophers": list(all_data.keys()),
    "total_passages": {phil: len(data["passages"]) for phil, data in all_data.items()}
}

with open('/content/rag_exports/config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("\n✅ Tous les fichiers exportés dans /content/rag_exports/")
print("\n📋 Fichiers à télécharger:")
!ls -lh /content/rag_exports/

## 10. Téléchargement ZIP

In [ ]:
# Créer archive ZIP pour téléchargement
!zip -r /content/rag_exports.zip /content/rag_exports/

# Télécharger via interface Colab
from google.colab import files
files.download('/content/rag_exports.zip')

print("\n✅ Archive téléchargée - À uploader sur Vast.ai pour Phase 1 !")

---

## 📝 Résumé

### ✅ Implémenté
- ✅ Parsing 3 corpus (Spinoza/Bergson/Kant) en passages
- ✅ Génération embeddings (sentence-transformers)
- ✅ Indexation FAISS par philosophe
- ✅ Retrieval sémantique fonctionnel
- ✅ Export index + passages pour réutilisation

### 📊 Statistiques
- **Modèle:** all-MiniLM-L6-v2 (dimension: 384)
- **Total passages:** ~XXX (à compléter après exécution)
- **Index FAISS:** 3 (Spinoza, Bergson, Kant)

### 📦 Fichiers Exportés
1. `spinoza_faiss.index` + `spinoza_passages.pkl`
2. `bergson_faiss.index` + `bergson_passages.pkl`
3. `kant_faiss.index` + `kant_passages.pkl`
4. `config.json` (metadata)

### ➡️ Prochaines Étapes
1. **Notebook 3:** Tests Mistral 7B (GPU Colab T4)
2. **Phase 1 (Vast.ai):** Upload index RAG + intégration complète

---

**💰 Coût:** 0€ (Colab gratuit GPU T4)

**⏱️ Temps:** ~1-2h (dépend taille corpus)

**🎯 Objectif Phase 0:** RAG sémantique prêt pour POC ✅